# Pandas
- 분석하려는 데이터는 대부분 시계열(Series)이거나 표(table) 형태로 정의해야 한다.
- Series 클래스와 DataFrame 클래스를 제공한다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Series 클래스
- numpy의 1차원 배열과 비슷하지만 각 데이터의 의미를 표시하는 인덱스를 붙일 수 있다.
- Series = index + value

In [2]:
# Series와 numpy array 비교
arr = np.array([1,2,3,4,'jslim'], dtype=np.object)
print(arr)
print(arr.dtype)

[1 2 3 4 'jslim']
object


In [3]:
s = pd.Series([1,2,3,4], dtype=np.float64)
print(s)
print(s.values)
print(type(s.values))
print(s.index)
print(type(s.index))

0    1.0
1    2.0
2    3.0
3    4.0
dtype: float64
[1. 2. 3. 4.]
<class 'numpy.ndarray'>
RangeIndex(start=0, stop=4, step=1)
<class 'pandas.core.indexes.range.RangeIndex'>


In [4]:
def serieInfo(s) :
    print('value : ', s.values)
    print('value type : ', type(s.values))
    print('index : ', s.index)
    print('index type : ', type(s.index))
    print('index + value : ', s)
    print(s)

In [5]:
# 인덱스의 라벨은 문자열뿐만 아니라 날짜, 시간, 정수 등 가능
s = pd.Series([344234, 23423, 3453, 45674],
              dtype=np.int32,
              index=['서울','부산','대전','광주'])
serieInfo(s)

value :  [344234  23423   3453  45674]
value type :  <class 'numpy.ndarray'>
index :  Index(['서울', '부산', '대전', '광주'], dtype='object')
index type :  <class 'pandas.core.indexes.base.Index'>
index + value :  서울    344234
부산     23423
대전      3453
광주     45674
dtype: int32
서울    344234
부산     23423
대전      3453
광주     45674
dtype: int32


In [6]:
s.index.name = '지역별'
print(s)

지역별
서울    344234
부산     23423
대전      3453
광주     45674
dtype: int32


In [7]:
# Series 연산
# index에 영향을 미치지 않는다
s/100

지역별
서울    3442.34
부산     234.23
대전      34.53
광주     456.74
dtype: float64

In [8]:
s/100000

지역별
서울    3.44234
부산    0.23423
대전    0.03453
광주    0.45674
dtype: float64

### series indexing

In [9]:
s['서울':'부산'] 
# label indexing 은 정수가 아니기 때문에 end - 1 을 하지 않는다.

지역별
서울    344234
부산     23423
dtype: int32

In [10]:
s[['서울','대전']] 
# 부분 indexing 은 대괄호를 하나 더 추가해준다.
# numpy와 비슷하지만 형식이 series

지역별
서울    344234
대전      3453
dtype: int32

In [11]:
'강원' in s

False

In [12]:
#  dictionary 데이터 타입처럼 사용 가능
for key, value in s.items() :
    print('key : {}, value = {}'.format(key, value))

key : 서울, value = 344234
key : 부산, value = 23423
key : 대전, value = 3453
key : 광주, value = 45674


In [13]:
s2 = pd.Series({'c':1, 'b':'5', 'a':-8, 'k':10}, dtype=np.float64)
serieInfo(s2)

value :  [ 1.  5. -8. 10.]
value type :  <class 'numpy.ndarray'>
index :  Index(['c', 'b', 'a', 'k'], dtype='object')
index type :  <class 'pandas.core.indexes.base.Index'>
index + value :  c     1.0
b     5.0
a    -8.0
k    10.0
dtype: float64
c     1.0
b     5.0
a    -8.0
k    10.0
dtype: float64


In [14]:
# Fancy Indexing & Boolean Indexing
print('fancy [0,2] indexing : {}'.format (s2[ [0,2] ]) )

# Boolean Indexing 2의 배수인 것
print('boolean s2%2==0 : {}'.format ( s2[s2%2==0] ))

fancy [0,2] indexing : c    1.0
a   -8.0
dtype: float64
boolean s2%2==0 : a    -8.0
k    10.0
dtype: float64


In [31]:
# index 의 라벨은 문자열뿐만 아니라 날짜, 시간, 정수 등 가능
s3 = pd.Series({'서울' : 344234, '부산' : 23423, '인천' : 3453343, '대전' : 3445674},
               dtype=np.int32,
               index=['광주','대전','부산','서울'])
serieInfo(s3)

value :  [     nan 3445674.   23423.  344234.]
value type :  <class 'numpy.ndarray'>
index :  Index(['광주', '대전', '부산', '서울'], dtype='object')
index type :  <class 'pandas.core.indexes.base.Index'>
index + value :  광주          NaN
대전    3445674.0
부산      23423.0
서울     344234.0
dtype: float64


In [46]:
diff_s = s - s2
print(diff_s)
# index가 다르기 때문에 NaN 값 나온다.

a    NaN
b    NaN
c    NaN
k    NaN
광주   NaN
대전   NaN
부산   NaN
서울   NaN
dtype: float64


### Series 를 활용한 날짜 데이터 응용
>- A 공장의 2019-01-01 부터 10 일간의 생산량을 Series 저장
- 생산량은 평균이 50 이고 편차가 5 인 정규분포 생성(정수)

>- B 공장의 2019-01-01 부터 10 일간의 생산량을 Series 저장
- 생샨량은 평균이 70 이고 편차가 8 인 정규분포 생성(정수)

>- 날짜별로 모든 공장의 생산량 합계를 구하기

In [34]:
import pandas as pd
import numpy as np
from datetime import date, datetime, timedelta
from dateutil.parser import parse

In [52]:
start_day = datetime(2019, 1, 1)
print(start_day)
facA = pd.Series([ int(x) for x in np.random.normal(50, 5, (10,))],
                index=[start_day + timedelta(days=x) for x in range(10)])
print(facA)
facB = pd.Series([ int(x) for x in np.random.normal(70, 8, (10,))],
                index=[start_day + timedelta(days=x) for x in range(10)])
print("*"*50)
print(facB)

2019-01-01 00:00:00
2019-01-01    48
2019-01-02    51
2019-01-03    52
2019-01-04    45
2019-01-05    50
2019-01-06    56
2019-01-07    55
2019-01-08    52
2019-01-09    54
2019-01-10    55
dtype: int64
**************************************************
2019-01-01    79
2019-01-02    67
2019-01-03    64
2019-01-04    70
2019-01-05    60
2019-01-06    72
2019-01-07    70
2019-01-08    72
2019-01-09    73
2019-01-10    76
dtype: int64


In [53]:
# 단위 테스트를 위한 셀
# type( np.random.normal(50, 5, (10,)))
print(start_day + timedelta(days=2))

2019-01-03 00:00:00


In [54]:
print(facA + facB)

2019-01-01    127
2019-01-02    118
2019-01-03    116
2019-01-04    115
2019-01-05    110
2019-01-06    128
2019-01-07    125
2019-01-08    124
2019-01-09    127
2019-01-10    131
dtype: int64
